# Create harmonized sgRNA guide annotation file for use with the CRISPR pipeline (2025)
This notebook describes the creation of a unified annotation file from the guide annotation files provided by the Hon, Huangfu, and Gersbach labs, according to the specification described in: https://github.com/pinellolab/CRISPR_Pipeline/blob/main/example_data/guide_metadata.tsv 

# Install libraries and set paths

In [5]:
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install seaborn
%pip install biomart

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [7]:
# Paths: TODO update if necessary
#local_path = "/cellar/users/aklie/data/datasets/tf_perturb_seq/ref/"
local_path = "C:/Users/seg95/Documents/tf_perturb_seq/"
#local_path = "D:/tf_perturb_seq/"

# Import merged guide reference file, along with guide index file

In [8]:
# Merged guide ref file
merged_guide_file = pd.read_csv(local_path + "outer_merged_file.csv")
merged_guide_file.head()

merged_guide_file_poolabcd = pd.read_csv(local_path + "outer_merged_file_poolabcd.csv")
merged_guide_file_poolf = pd.read_csv(local_path + "outer_merged_file_poolf.csv")

In [9]:
# sgRNA index files
sgrna_index_poolabcd = pd.read_csv(local_path + "sgRNA_index_v0.csv", sep = "\t")
sgrna_index_poolf = pd.read_csv(local_path + "igvf_poolF_annotation.csv", sep = "\t")

sgrna_index_dacc_annot = pd.read_csv(local_path + "sgRNA_index_dacc_annot_reference.csv", sep = "\t")
print(len(set(sgrna_index_dacc_annot['protospacer']).intersection(set(merged_guide_file_poolabcd['protospacer']))))

def adjust_index_file(sgrna_index, name_sgrna_seq = 'sgRNA_seq', add_leading_G = True):
    if(name_sgrna_seq == "sgRNA_seq"):
        sgrna_index['strand'] = sgrna_index['target_loc'].str.extract(r'\((\+|\-)\)')
        sgrna_index['oligo'] = sgrna_index['oligo'].str.upper()
    else:
        sgrna_index['oligo_sequence'] = sgrna_index['oligo_sequence'].str.upper()
    sgrna_index[name_sgrna_seq] = sgrna_index[name_sgrna_seq].str.upper()
    # Adjust the index file to add leading Gs if needed
    if(add_leading_G):
        sgrna_index[name_sgrna_seq] = 'G' + sgrna_index[name_sgrna_seq]
    return sgrna_index

sgrna_index_poolabcd = adjust_index_file(sgrna_index_poolabcd)
sgrna_index_poolf = adjust_index_file(sgrna_index_poolf, name_sgrna_seq= 'protospacer', add_leading_G = False)

sgrna_index_dacc_annot['protospacer'] = sgrna_index_dacc_annot['protospacer'].str.upper()
sgrna_index_poolf['protospacer'] = sgrna_index_poolf['protospacer'].str.upper()
#sgrna_index_dacc_annot['protospacer'] = [s[1:] if len(s) > 0 else s for s in sgrna_index_dacc_annot['protospacer']]
#sgrna_index_dacc_annot['reverse_compliment'] = sgrna_index_dacc_annot['reverse_compliment'].str.rstrip('C')

13188


In [10]:
# Add a reverse compliment if needed
def reverse_compliment(sequence):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return "".join(complement.get(base, base) for base in reversed(sequence.upper()))

sgrna_index_poolabcd['reverse_compliment'] = sgrna_index_poolabcd['sgRNA_seq'].apply(reverse_compliment)
sgrna_index_poolf.rename(columns={"antisense_sequence": "reverse_compliment"})

print("Index:")
print(sgrna_index_poolabcd.head())
print(sgrna_index_poolf.head())
print("Annot:")
print(sgrna_index_dacc_annot.head())

Index:
                   target_loc              element_seq     target source  \
0  chr17:36948966-36948984(+)  chr17:36948966-36949088  AATF_P1P2     TF   
1  chr17:36949026-36949044(+)  chr17:36948966-36949088  AATF_P1P2     TF   
2  chr17:36949013-36949031(+)  chr17:36948966-36949088  AATF_P1P2     TF   
3  chr17:36949070-36949088(-)  chr17:36948966-36949088  AATF_P1P2     TF   
4  chr17:36949031-36949049(+)  chr17:36948966-36949088  AATF_P1P2     TF   

              sgRNA_seq                                              oligo  \
0  GAGTGGCCGGTCCAGAGCTG  GTGGAAAGGACGAAACACCGAGTGGCCGGTCCAGAGCTGGTTTAAG...   
1  GGGATCAAGGCGAGAGGATC  GTGGAAAGGACGAAACACCGGGATCAAGGCGAGAGGATCGTTTAAG...   
2  GGAGTCGGGGAATCGGATCA  GTGGAAAGGACGAAACACCGGAGTCGGGGAATCGGATCAGTTTAAG...   
3  GAAATGTGCGGCCCAACCCC  GTGGAAAGGACGAAACACCGAAATGTGCGGCCCAACCCCGTTTAAG...   
4  GAAGGCGAGAGGATCCGGCA  GTGGAAAGGACGAAACACCGAAGGCGAGAGGATCCGGCAGTTTAAG...   

  gene_target chr_target  chr_start_target  chr_end_target chr_elem

In [11]:
sgrna_index_dacc_annot["protospacer_upper"] = sgrna_index_dacc_annot["protospacer"].str.upper() 

print(len(set(sgrna_index_poolabcd['sgRNA_seq']).intersection(sgrna_index_dacc_annot['protospacer_upper'])))

13470


In [12]:
# Merge pool A-D index and DACC files into one; pool F file has sufficient info for matching
sgrna_index_merged = pd.merge(
    sgrna_index_dacc_annot,
    sgrna_index_poolabcd,
    left_on=['protospacer_upper', 'reverse_compliment'],
    right_on=['sgRNA_seq', 'reverse_compliment'],
    how="outer"
)
print(sgrna_index_merged.head())
print(sgrna_index_merged.shape)

               protospacer_ID           protospacer intended_target_name  \
0                     OR5K2-2  GAAAAAATTGTAGAGGAATA                OR5K2   
1    SP1_+_53773993.23-P1P2-1  GAAAAACGCGGACGCTGACG                  SP1   
2    SP8_-_20826141.23-P1P2-2  GAAAAAGATCCTCTGAGAGG                  SP8   
3    FOXN3_-_89883583.23-P2-1  GAAAAAGGCGACACATGACC                FOXN3   
4  ZNF85_+_21106076.23-P1P2-1  GAAAACAAGACCTAGAGCTC                ZNF85   

        type genomic_element    reverse_compliment     protospacer_upper  \
0  targeting        promoter  TATTCCTCTACAATTTTTTC  GAAAAAATTGTAGAGGAATA   
1  targeting        promoter  CGTCAGCGTCCGCGTTTTTC  GAAAAACGCGGACGCTGACG   
2  targeting        promoter  CCTCTCAGAGGATCTTTTTC  GAAAAAGATCCTCTGAGAGG   
3  targeting        promoter  GGTCATGTGTCGCCTTTTTC  GAAAAAGGCGACACATGACC   
4  targeting        promoter  GAGCTCTAGGTCTTGTTTTC  GAAAACAAGACCTAGAGCTC   

                   target_loc              element_seq      target  ...  \
0          

# Reformat to resemble input to the CRISPR pipeline

In [13]:
# Import example file for the CRISPR pipeline
example_crispr_file = pd.read_csv(local_path + "crispr_annot_sample.tsv", sep = "\t")

In [14]:
example_crispr_file.head()

,guide_id,spacer,targeting,type,guide_chr,guide_start,guide_end,strand,pam,intended_target_name,intended_target_chr,intended_target_start,intended_target_end
0,AFF4_sg1,CCAGCGGACGGGGCGGGGAC,True,targeting,chr5,132299282.0,132299302.0,-,NGG,AFF4,chr5,132875395.0,132963634.0
1,AFF4_sg2,CCGCCAGCGGACGGGGCGGC,True,targeting,chr5,132299282.0,132299302.0,-,NGG,AFF4,chr5,132875395.0,132963634.0
2,AFF4_sg3,CGTCCGCTGGCGGCGGCGAC,True,targeting,chr5,132299252.0,132299272.0,-,NGG,AFF4,chr5,132875395.0,132963634.0
3,AFF4_sg4,CTGCGTCAGTCACAGCCCTC,True,targeting,chr5,132299279.0,132299299.0,-,NGG,AFF4,chr5,132875395.0,132963634.0
4,AFF4_sg5,GCGGACGGGGCGGGGATCCC,True,targeting,chr5,132299279.0,132299299.0,-,NGG,AFF4,chr5,132875395.0,132963634.0


In [15]:
# Keep only necessary columns and reorder them to match 
def prune_and_rename_cols(merged_guide_file, is_pool_f=False):
        ref_clean_sub = merged_guide_file[['protospacer', 'type', 'reverse_compliment']].copy()
        if(is_pool_f):
                ref_clean_sub["guide_id"] = (
                        merged_guide_file['id_gersbach'].combine_first(merged_guide_file['id_engreitz'])
                )
                ref_clean_sub["intended_target_name"] = (
                        merged_guide_file['intended_target_name_gersbach'].combine_first(merged_guide_file['intended_target_name_engreitz'])
                )
        else:
                ref_clean_sub["guide_id"] = (
                        merged_guide_file['id_hon']
                        .combine_first(merged_guide_file['id_gersbach'])
                        .combine_first(merged_guide_file['id_engreitz'])
                        .combine_first(merged_guide_file['id_huangfu'])
                )
                ref_clean_sub["intended_target_name"] = (
                        merged_guide_file['intended_target_name_hon']
                        .combine_first(merged_guide_file['intended_target_name_gersbach'])
                        .combine_first(merged_guide_file['intended_target_name_engreitz'])
                        .combine_first(merged_guide_file['intended_target_name_huangfu'])
                )
                #print(ref_clean_sub.head())

        # For each, use only one ID - use Hon lab as a reference, but fill will Gersbach/Engreitz where values are missing
        ref_clean_sub = ref_clean_sub.rename(columns={'protospacer': 'spacer'})
        ref_clean_sub = ref_clean_sub[['guide_id', 'spacer', 'type', 'intended_target_name', 'reverse_compliment']]
        print(ref_clean_sub.head())
        return ref_clean_sub

# Call function
ref_clean_sub = prune_and_rename_cols(merged_guide_file)
ref_clean_sub_poolabcd = prune_and_rename_cols(merged_guide_file_poolabcd)
ref_clean_sub_poolf = prune_and_rename_cols(merged_guide_file_poolf, is_pool_f=True)

                       guide_id                spacer       type  \
0    FOXN1_-_26833391.23-P1P2-1  GCACAGGACGGCCGAGCTGA  targeting   
1     EN2_-_155251011.23-P1P2-1  GCTCCGTGTGCGCCGCGGGA  targeting   
2  BCLAF1_-_136610510.23-P1P2-2  GCTCCGTTGCAACCACACAG  targeting   
3      KLF6_-_3827130.23-P1P2-2  GCTGGAGGATCGATCGGCGG  targeting   
4     ELF1_+_41593362.23-P1P2-2  GTGAGCTGATAAACAGAGGG  targeting   

  intended_target_name    reverse_compliment  
0                FOXN1  TCAGCTCGGCCGTCCTGTGC  
1                  EN2  TCCCGCGGCGCACACGGAGC  
2               BCLAF1  CTGTGTGGTTGCAACGGAGC  
3                 KLF6  CCGCCGATCGATCCTCCAGC  
4                 ELF1  CCCTCTGTTTATCAGCTCAC  
                     guide_id                spacer              type  \
0  TFEC_-_115670779.23-P1P2-1  GCATATGCACCATGCCAGAA         targeting   
1  NR2C1_-_95467292.23-P1P2-2  GGATGTGGGATCGAGATTCA         targeting   
2   NANOG_+_7942459.23-P1P2-2  GTTTTTCCATTATAACTTGG         targeting   
3                

In [16]:
# Add 'targeting' column; if type == targeting, set to True, otherwise False
def check_targeting(value):
    if(value == "targeting"):
        return True
    else:
        return False

def add_targeting_col(ref_clean_sub):
    ref_clean_sub['targeting'] = ref_clean_sub['type'].apply(check_targeting)
    order = ['guide_id', 'spacer', 'targeting', 'type', 'intended_target_name']
    ref_clean_sub = ref_clean_sub[order]
    print(ref_clean_sub.head())
    return ref_clean_sub

ref_clean_sub = add_targeting_col(ref_clean_sub)
ref_clean_sub_poolabcd = add_targeting_col(ref_clean_sub_poolabcd)
ref_clean_sub_poolf = add_targeting_col(ref_clean_sub_poolf)

                       guide_id                spacer  targeting       type  \
0    FOXN1_-_26833391.23-P1P2-1  GCACAGGACGGCCGAGCTGA       True  targeting   
1     EN2_-_155251011.23-P1P2-1  GCTCCGTGTGCGCCGCGGGA       True  targeting   
2  BCLAF1_-_136610510.23-P1P2-2  GCTCCGTTGCAACCACACAG       True  targeting   
3      KLF6_-_3827130.23-P1P2-2  GCTGGAGGATCGATCGGCGG       True  targeting   
4     ELF1_+_41593362.23-P1P2-2  GTGAGCTGATAAACAGAGGG       True  targeting   

  intended_target_name  
0                FOXN1  
1                  EN2  
2               BCLAF1  
3                 KLF6  
4                 ELF1  
                     guide_id                spacer  targeting  \
0  TFEC_-_115670779.23-P1P2-1  GCATATGCACCATGCCAGAA       True   
1  NR2C1_-_95467292.23-P1P2-2  GGATGTGGGATCGAGATTCA       True   
2   NANOG_+_7942459.23-P1P2-2  GTTTTTCCATTATAACTTGG       True   
3                     OR8B3-5  GTTTTTGTCTTCAAAAATCT      False   
4  ZNF48_+_30406782.23-P1P2-1  GCTCCGCGCCAAGC

In [17]:
# Add PAM
def add_pam(ref_clean_sub):
    ref_clean_sub['pam'] = 'NGG'
    print(ref_clean_sub.head())
    print(ref_clean_sub.shape)
    return ref_clean_sub

ref_clean_sub = add_pam(ref_clean_sub)
ref_clean_sub_poolabcd = add_pam(ref_clean_sub_poolabcd)
ref_clean_sub_poolf = add_pam(ref_clean_sub_poolf)

                       guide_id                spacer  targeting       type  \
0    FOXN1_-_26833391.23-P1P2-1  GCACAGGACGGCCGAGCTGA       True  targeting   
1     EN2_-_155251011.23-P1P2-1  GCTCCGTGTGCGCCGCGGGA       True  targeting   
2  BCLAF1_-_136610510.23-P1P2-2  GCTCCGTTGCAACCACACAG       True  targeting   
3      KLF6_-_3827130.23-P1P2-2  GCTGGAGGATCGATCGGCGG       True  targeting   
4     ELF1_+_41593362.23-P1P2-2  GTGAGCTGATAAACAGAGGG       True  targeting   

  intended_target_name  pam  
0                FOXN1  NGG  
1                  EN2  NGG  
2               BCLAF1  NGG  
3                 KLF6  NGG  
4                 ELF1  NGG  
(19956, 6)
                     guide_id                spacer  targeting  \
0  TFEC_-_115670779.23-P1P2-1  GCATATGCACCATGCCAGAA       True   
1  NR2C1_-_95467292.23-P1P2-2  GGATGTGGGATCGAGATTCA       True   
2   NANOG_+_7942459.23-P1P2-2  GTTTTTCCATTATAACTTGG       True   
3                     OR8B3-5  GTTTTTGTCTTCAAAAATCT      False   
4  Z

In [18]:
print(sgrna_index_merged.head())
print(len(set(sgrna_index_merged['protospacer_upper']).intersection(set(ref_clean_sub_poolabcd['spacer']))))
print(len(set(sgrna_index_poolf['protospacer']).intersection(set(ref_clean_sub_poolf['spacer']))))  

               protospacer_ID           protospacer intended_target_name  \
0                     OR5K2-2  GAAAAAATTGTAGAGGAATA                OR5K2   
1    SP1_+_53773993.23-P1P2-1  GAAAAACGCGGACGCTGACG                  SP1   
2    SP8_-_20826141.23-P1P2-2  GAAAAAGATCCTCTGAGAGG                  SP8   
3    FOXN3_-_89883583.23-P2-1  GAAAAAGGCGACACATGACC                FOXN3   
4  ZNF85_+_21106076.23-P1P2-1  GAAAACAAGACCTAGAGCTC                ZNF85   

        type genomic_element    reverse_compliment     protospacer_upper  \
0  targeting        promoter  TATTCCTCTACAATTTTTTC  GAAAAAATTGTAGAGGAATA   
1  targeting        promoter  CGTCAGCGTCCGCGTTTTTC  GAAAAACGCGGACGCTGACG   
2  targeting        promoter  CCTCTCAGAGGATCTTTTTC  GAAAAAGATCCTCTGAGAGG   
3  targeting        promoter  GGTCATGTGTCGCCTTTTTC  GAAAAAGGCGACACATGACC   
4  targeting        promoter  GAGCTCTAGGTCTTGTTTTC  GAAAACAAGACCTAGAGCTC   

                   target_loc              element_seq      target  ...  \
0          

In [19]:
# Check for repeated spacer sequences in index file
print(sgrna_index_merged['protospacer_upper'].value_counts().loc[lambda x: x > 1])

protospacer_upper
GCAGCCACGCGAGAGTAGAA    3
GATTGTCACGGCACATTCCA    2
GCCTGCGGGCGGGACAGAGG    2
GCAATGGTTAGGCTCTTACA    2
GAACACCGGATGTGGGGGAG    2
                       ..
GTGGGGAGGAAGCGGTTCTA    2
GTGGTGCGACCACCACACCC    2
GTGTCGGAGGACGAGGACCG    2
GTTCCCGGTTCGCTCGGCCG    2
GTTCCGCTCCAGGGAAGAGG    2
Name: count, Length: 92, dtype: int64


In [20]:
# Remove multiple mappings from sgrna_index_merged
def deduplicate_index_file(df):
    def chrom_rank(chrom):
        if pd.isna(chrom):
            return 100
        if isinstance(chrom, str) and chrom.startswith("chr"):
            c = chrom[3:]
            if c.isdigit():
                return int(c)
            elif c == "X":
                return 23
            elif c == "Y":
                return 24
        return 100  # fallback
    
    df = df.copy()
    
    # Rank and sorting
    df["chrom_rank"] = df["chr_target"].map(chrom_rank)
    df["sort_key"] = (
        df["chrom_rank"].fillna(100) * 1e12 +
        df["chr_start_target"].fillna(0) * 1e6 +
        (df["chr_end_target"].fillna(0) - df["chr_start_target"].fillna(0))
    )
    
    # Group by spacer sequence
    grouped = df.groupby("protospacer_upper", group_keys=False)
    
    # Keep only groups where all key columns are the same across rows
    key_cols = [
        "chr_target", "chr_start_target", "chr_end_target",
        "chr_element", "chr_start_element", "chr_end_element"
    ]
    
    def is_consistent(group):
        return all(group[col].nunique(dropna=False) == 1 for col in key_cols)
    
    consistent_df = grouped.filter(is_consistent)
    
    # Deduplicate remaining consistent rows by keeping best ranked
    deduped_df = (
        consistent_df.sort_values("sort_key")
                     .drop_duplicates(subset="protospacer_upper", keep="first")
                     .drop(columns=["chrom_rank", "sort_key"])
    )
    
    return deduped_df

# Apply deduplication before merging
sgrna_index_merged = deduplicate_index_file(sgrna_index_merged)

In [21]:
# Add the 'guide_chr', 'guide_start', and 'guide_end' values, which are given as 'chr_target', 'chr_start_target', 'chr_end_target', and 'strand'
def add_guide_coords(ref_clean_sub, sgrna_index_merged):
    ref_clean_sub = pd.merge(
        ref_clean_sub,
        sgrna_index_merged[['protospacer_upper', 'chr_target', 'chr_start_target', 'chr_end_target', 'strand']],
        left_on='spacer',
        right_on='protospacer_upper',
        how='left'
    )
    # Remove protospacer_upper column
    ref_clean_sub = ref_clean_sub.drop(columns=['protospacer_upper'])
    # Rename intended guide names
    ref_clean_sub.rename(columns={'chr_target': 'guide_chr', 
                                  'chr_start_target': 'guide_start',
                                  'chr_end_target': 'guide_end'},
                                  inplace=True)


    print(ref_clean_sub.head())
    return ref_clean_sub


ref_clean_sub_poolabcd = add_guide_coords(ref_clean_sub_poolabcd, sgrna_index_merged)
print(ref_clean_sub_poolabcd.head())

# Columns are already correctly labeled for pool F
ref_clean_sub_poolf = pd.merge(
    ref_clean_sub_poolf,
    sgrna_index_poolf[['protospacer', 'guide_chr', 'guide_start', 'guide_end', 'strand']],
    left_on='spacer',
    right_on='protospacer',
    how='left'
)
print(ref_clean_sub_poolf.head())

                     guide_id                spacer  targeting  \
0  TFEC_-_115670779.23-P1P2-1  GCATATGCACCATGCCAGAA       True   
1  NR2C1_-_95467292.23-P1P2-2  GGATGTGGGATCGAGATTCA       True   
2   NANOG_+_7942459.23-P1P2-2  GTTTTTCCATTATAACTTGG       True   
3                     OR8B3-5  GTTTTTGTCTTCAAAAATCT      False   
4  ZNF48_+_30406782.23-P1P2-1  GCTCCGCGCCAAGCCGGGAG       True   

               type intended_target_name  pam guide_chr  guide_start  \
0         targeting                 TFEC  NGG      chr7  116030705.0   
1         targeting                NR2C1  NGG     chr12   95073493.0   
2         targeting                NANOG  NGG     chr12    7789912.0   
3  negative_control                OR8B3  NGG       NaN          NaN   
4         targeting                ZNF48  NGG     chr16   30395465.0   

     guide_end strand  
0  116030723.0      +  
1   95073511.0      +  
2    7789930.0      +  
3          NaN    NaN  
4   30395483.0      -  
                     guide

In [22]:
# Add the intended_target_chr/intended_target_start/intended_target_end values, which are given as 'chr_element', 'chr_start_element', 'chr_end_element'
# Note that this refers to the element being targeted, not the gene itself
def add_element_coords(ref_clean_sub, sgrna_index_merged):
    ref_clean_sub = pd.merge(
        ref_clean_sub,
        sgrna_index_merged[['protospacer_upper', 'chr_element', 'chr_start_element', 'chr_end_element']],
        left_on='spacer',
        right_on='protospacer_upper',
        how='left'
    )
    # Remove protospacer_upper column
    ref_clean_sub = ref_clean_sub.drop(columns=['protospacer_upper'])
    # Rename intended target names
    ref_clean_sub.rename(columns={'chr_element': 'intended_target_chr', 
                                  'chr_start_element': 'intended_target_start',
                                  'chr_end_element': 'intended_target_end'},
                                  inplace=True)
    print(ref_clean_sub.head())
    return ref_clean_sub


ref_clean_sub_poolabcd = add_element_coords(ref_clean_sub_poolabcd, sgrna_index_merged)
ref_clean_sub_poolabcd.head()

# Columns are already correctly labeled for pool F
ref_clean_sub_poolf = pd.merge(
    ref_clean_sub_poolf,
    sgrna_index_poolf[['protospacer', 'intended_target_chr', 'intended_target_start', 'intended_target_end']],
    left_on='spacer',
    right_on='protospacer',
    how='left'
)
print(ref_clean_sub_poolf.head())

                     guide_id                spacer  targeting  \
0  TFEC_-_115670779.23-P1P2-1  GCATATGCACCATGCCAGAA       True   
1  NR2C1_-_95467292.23-P1P2-2  GGATGTGGGATCGAGATTCA       True   
2   NANOG_+_7942459.23-P1P2-2  GTTTTTCCATTATAACTTGG       True   
3                     OR8B3-5  GTTTTTGTCTTCAAAAATCT      False   
4  ZNF48_+_30406782.23-P1P2-1  GCTCCGCGCCAAGCCGGGAG       True   

               type intended_target_name  pam guide_chr  guide_start  \
0         targeting                 TFEC  NGG      chr7  116030705.0   
1         targeting                NR2C1  NGG     chr12   95073493.0   
2         targeting                NANOG  NGG     chr12    7789912.0   
3  negative_control                OR8B3  NGG       NaN          NaN   
4         targeting                ZNF48  NGG     chr16   30395465.0   

     guide_end strand intended_target_chr  intended_target_start  \
0  116030723.0      +                chr7            116030682.0   
1   95073511.0      +             

In [23]:
print(example_crispr_file.head())

# Reorganize columns to match
new_order = ['guide_id', 'spacer', 'targeting', 'type', 'guide_chr', 'guide_start', 'guide_end', 'strand', 'pam', 'intended_target_name', 'intended_target_chr', 'intended_target_start', 'intended_target_end']
ref_clean_sub_poolabcd = ref_clean_sub_poolabcd[new_order].drop_duplicates()
print(ref_clean_sub_poolabcd.head())
ref_clean_sub_poolf = ref_clean_sub_poolf[new_order].drop_duplicates()
print(ref_clean_sub_poolf.head())


   guide_id                spacer  targeting       type guide_chr  \
0  AFF4_sg1  CCAGCGGACGGGGCGGGGAC       True  targeting      chr5   
1  AFF4_sg2  CCGCCAGCGGACGGGGCGGC       True  targeting      chr5   
2  AFF4_sg3  CGTCCGCTGGCGGCGGCGAC       True  targeting      chr5   
3  AFF4_sg4  CTGCGTCAGTCACAGCCCTC       True  targeting      chr5   
4  AFF4_sg5  GCGGACGGGGCGGGGATCCC       True  targeting      chr5   

   guide_start    guide_end strand  pam intended_target_name  \
0  132299282.0  132299302.0      -  NGG                 AFF4   
1  132299282.0  132299302.0      -  NGG                 AFF4   
2  132299252.0  132299272.0      -  NGG                 AFF4   
3  132299279.0  132299299.0      -  NGG                 AFF4   
4  132299279.0  132299299.0      -  NGG                 AFF4   

  intended_target_chr  intended_target_start  intended_target_end  
0                chr5            132875395.0          132963634.0  
1                chr5            132875395.0          132963634.

In [24]:
# Remove any rows with non-standard chromosomes (i.e. chr1, chr2, chrX, etc., not chrU) by making sure chr is not followed by a letter other than X or Y
standard_chr_pattern = r'^chr(\d+|X|Y)$'

ref_clean_sub_poolabcd = ref_clean_sub_poolabcd[
    ref_clean_sub_poolabcd['guide_chr'].notna() &
    ref_clean_sub_poolabcd['intended_target_chr'].notna() &
    ref_clean_sub_poolabcd['guide_chr'].str.match(standard_chr_pattern) &
    ref_clean_sub_poolabcd['intended_target_chr'].str.match(standard_chr_pattern)
]
print(ref_clean_sub_poolabcd.shape)
ref_clean_sub_poolf = ref_clean_sub_poolf[
    ref_clean_sub_poolf['guide_chr'].notna() &
    ref_clean_sub_poolf['intended_target_chr'].notna() &
    ref_clean_sub_poolf['guide_chr'].str.match(standard_chr_pattern) &
    ref_clean_sub_poolf['intended_target_chr'].str.match(standard_chr_pattern)
]
print(ref_clean_sub_poolf.shape)

(13426, 13)
(2702, 13)


In [27]:
# Also write a version without mostly NA values, duplicate rows
#print(ref_clean_sub_poolabcd.shape)
#ref_clean_sub_poolabcd_clean = ref_clean_sub_poolabcd.dropna(thresh = (len(ref_clean_sub_poolabcd.columns)/2)).drop_duplicates()
#print(ref_clean_sub_poolabcd_clean.shape)
#ref_clean_sub_poolabcd_clean.to_csv(local_path + "harmonized_guide_file_poolabcd_nomissing.csv")

In [28]:
# Create a merged Pool ABCD and Pool F file
ref_clean_sub_poolabcdf = pd.concat([ref_clean_sub_poolabcd, ref_clean_sub_poolf]).drop_duplicates()
print(ref_clean_sub_poolabcdf.head())
print(ref_clean_sub_poolabcdf.shape)

                     guide_id                spacer  targeting       type  \
0  TFEC_-_115670779.23-P1P2-1  GCATATGCACCATGCCAGAA       True  targeting   
1  NR2C1_-_95467292.23-P1P2-2  GGATGTGGGATCGAGATTCA       True  targeting   
2   NANOG_+_7942459.23-P1P2-2  GTTTTTCCATTATAACTTGG       True  targeting   
4  ZNF48_+_30406782.23-P1P2-1  GCTCCGCGCCAAGCCGGGAG       True  targeting   
5   TIGD7_-_3355365.23-P1P2-2  GCTTCCCCGGCGGTAGGGAA       True  targeting   

  guide_chr  guide_start    guide_end strand  pam intended_target_name  \
0      chr7  116030705.0  116030723.0      +  NGG                 TFEC   
1     chr12   95073493.0   95073511.0      +  NGG                NR2C1   
2     chr12    7789912.0    7789930.0      +  NGG                NANOG   
4     chr16   30395465.0   30395483.0      -  NGG                ZNF48   
5     chr16    3305376.0    3305394.0      -  NGG                TIGD7   

  intended_target_chr  intended_target_start  intended_target_end  
0                chr7   

In [29]:
# Interrogate duplicate spacers
duplicate_spacers = ref_clean_sub_poolabcdf[ref_clean_sub_poolabcdf.duplicated(subset=['spacer'])]
print(duplicate_spacers.head())
print(duplicate_spacers.shape)

# Find the columns that are different between the duplicate spacers
duplicate_spacers_diff = duplicate_spacers.loc[:, duplicate_spacers.nunique() > 1]
print(duplicate_spacers_diff.head())
print(duplicate_spacers_diff.shape)

# Remove duplicate spacers
#ref_clean_sub_poolabcdf = ref_clean_sub_poolabcdf.drop_duplicates(subset=['spacer'])
#print(ref_clean_sub_poolabcdf.head())
#print(ref_clean_sub_poolabcdf.shape)


         guide_id               spacer  targeting       type guide_chr  \
37    LOC541473.5  GTGGCGTGAGCCTTCGGCC       True  targeting      chr7   
59   DPY30_P1P2.2  GTGCGACAGCCGTCACTTA       True  targeting      chr2   
99          NCF1B  GTCCTTCCCGTCGCCTGCA       True  targeting      chr7   
100         NCF1B  GTCCTTCCCGTCGCCTGCA       True  targeting      chr7   
101         NCF1B  GTCCTTCCCGTCGCCTGCA       True  targeting      chr7   

     guide_start   guide_end strand  pam intended_target_name  \
37    72969776.0  72969795.0      -  NGG            LOC541473   
59    32039805.0  32039823.0      -  NGG                DPY30   
99    73220402.0  73220421.0      -  NGG                NCF1B   
100   75172167.0  75172186.0      +  NGG                NCF1B   
101   75172167.0  75172186.0      +  NGG                NCF1B   

    intended_target_chr  intended_target_start  intended_target_end  
37                 chr7             72969431.0           72969931.0  
59                 chr2 

In [ ]:
# Fix any Excel-style gene names converted to dates
import re
month_gene_map = {
    "JAN": "JAN",
    "FEB": "FEB",
    "MAR": "MARCH",
    "APR": "APR",
    "MAY": "MAY",
    "JUN": "JUN",
    "JUL": "JUL",
    "AUG": "AUG",
    "SEP": "SEPT",
    "OCT": "OCT",
    "NOV": "NOV",
    "DEC": "DEC"
}

def fix_excel_date_genes(symbol):
    """Convert Excel-mangled gene symbols like 5-SEP to SEPT5."""
    m = re.match(r"^(\d{1,2})-([A-Z]{3})$", symbol)
    if m and m.group(2) in month_gene_map:
        num, month = m.groups()
        return f"{month_gene_map[month]}{num}"
    return symbol

ref_clean_sub_poolabcdf['intended_target_name'] = ref_clean_sub_poolabcdf['intended_target_name'].apply(fix_excel_date_genes)
ref_clean_sub_poolabcd['intended_target_name'] = ref_clean_sub_poolabcd['intended_target_name'].apply(fix_excel_date_genes)
ref_clean_sub_poolf['intended_target_name'] = ref_clean_sub_poolf['intended_target_name'].apply(fix_excel_date_genes)

In [26]:
# Write to file
ref_clean_sub_poolabcd.to_csv(local_path + "harmonized_guide_file_poolabcd.csv")
ref_clean_sub_poolabcd.to_csv(
    local_path + "harmonized_guide_file_poolabcd.tsv",
    sep='\t',
    index=False,
    lineterminator='\n'
)
ref_clean_sub_poolf.to_csv(local_path + "harmonized_guide_file_poolf.csv")
ref_clean_sub_poolf.to_csv(
    local_path + "harmonized_guide_file_poolf.tsv",
    sep='\t',
    index=False,
    lineterminator='\n'
)

In [30]:
ref_clean_sub_poolabcdf.to_csv(local_path + "harmonized_guide_file_poolabcdf.csv")

# Write to tsv file, including header
ref_clean_sub_poolabcdf.to_csv(
    local_path + "harmonized_guide_file_poolabcdf.tsv",
    sep='\t',
    index=False,
    lineterminator='\n'
)

---

In [33]:
%pip install pybiomart

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
# Convert intended_target_name to Ensembl ID using pyBiomart
from pybiomart import Dataset

dataset = Dataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')

# Fetch mapping
mapping = dataset.query(attributes=['hgnc_symbol', 'ensembl_gene_id', 'external_synonym'])
mapping.columns = ['intended_target_name', 'ensembl_gene_id', 'external_synonym']
mapping.head()

,intended_target_name,ensembl_gene_id,external_synonym
0,MT-TF,ENSG00000210049,MTTF
1,MT-TF,ENSG00000210049,TRNF
2,MT-RNR1,ENSG00000211459,12S
3,MT-RNR1,ENSG00000211459,MOTS-C
4,MT-RNR1,ENSG00000211459,MTRNR1


In [40]:
# Combine HGNC symbol and synonyms into a single mapping dataframe
# Melt external_synonym if it's a comma-separated list
mapping_expanded = mapping.copy()
mapping_expanded['external_synonym'] = mapping_expanded['external_synonym'].fillna('')
mapping_expanded = mapping_expanded.assign(
    synonym_list=mapping_expanded['external_synonym'].str.split(',')
).explode('synonym_list')
mapping_expanded.head()

,intended_target_name,ensembl_gene_id,external_synonym,synonym_list
0,MT-TF,ENSG00000210049,MTTF,MTTF
1,MT-TF,ENSG00000210049,TRNF,TRNF
2,MT-RNR1,ENSG00000211459,12S,12S
3,MT-RNR1,ENSG00000211459,MOTS-C,MOTS-C
4,MT-RNR1,ENSG00000211459,MTRNR1,MTRNR1


In [55]:
# Combine intended_target_name and synonym_list into one lookup table
lookup = pd.concat([
    mapping_expanded[['intended_target_name', 'ensembl_gene_id']].rename(columns={'intended_target_name': 'symbol'}),
    mapping_expanded[['synonym_list', 'ensembl_gene_id']].rename(columns={'synonym_list': 'symbol'})
]).drop_duplicates()
#print(lookup.head())
lookup['symbol'] = lookup['symbol'].apply(lambda x: str(x).upper().replace('-', '').replace('_',''))
print(lookup.head())

     symbol  ensembl_gene_id
0     MT-TF  ENSG00000210049
2   MT-RNR1  ENSG00000211459
5     MT-TV  ENSG00000210077
7   MT-RNR2  ENSG00000210082
10   MT-TL1  ENSG00000209082
    symbol  ensembl_gene_id
0     MTTF  ENSG00000210049
2   MTRNR1  ENSG00000211459
5     MTTV  ENSG00000210077
7   MTRNR2  ENSG00000210082
10   MTTL1  ENSG00000209082


In [51]:
def clean_symbol(s):
    s = str(s).upper()
    # Remove dashes for relaxed matching
    s = s.replace('-', '').replace('_', '')
    return s

In [56]:
# Merge with data frame and replace intended_target_name with Ensembl IDs
def replace_w_ensembl(ref_clean, mapping):

    # Make all symbols uppercase for matching
    ref_clean = ref_clean.copy()
    ref_clean['intended_target_name'] = ref_clean['intended_target_name'].apply(clean_symbol)

    mapping = mapping.copy()
    mapping['symbol'] = mapping['symbol'].apply(clean_symbol)

    ref_clean = ref_clean.merge(mapping, left_on='intended_target_name',
                                right_on='symbol', how='left')

    # For missing rows, try matching external_synonym
    #missing_mask = ref_clean['ensembl_gene_id'].isna()
    #if missing_mask.any():
    #    # Merge missing rows against exploded synonyms
    #    ref_clean_missing = ref_clean[missing_mask].merge(
    #        mapping[['synonym_list', 'ensembl_gene_id']],
    #        left_on='intended_target_name',
    #        right_on='synonym_list',
    #        how='left'
    #    )
    #   # Fill Ensembl IDs where found
    #    print(ref_clean_missing.head())
    #    ref_clean.loc[missing_mask, 'ensembl_gene_id'] = ref_clean_missing['ensembl_gene_id'].values

    # Identify still-missing mappings
    missing_mask = ref_clean['ensembl_gene_id'].isna()
    num_missing = missing_mask.sum()
    missing_genes = ref_clean.loc[missing_mask, 'intended_target_name'].unique()

    # Print summary
    print(f"Number of rows with missing Ensembl mapping: {num_missing}")
    print(f"Gene symbols with no mapping:\n{missing_genes}")

    # Keep only rows with a mapping
    ref_clean = ref_clean[~missing_mask].copy()

    # Replace intended_target_name with Ensembl ID
    ref_clean['intended_target_name'] = ref_clean['ensembl_gene_id']

    # Drop temporary columns
    ref_clean.drop(columns=['ensembl_gene_id'], inplace=True)

    return ref_clean


ref_clean_sub_poolabcdf_ensembl = replace_w_ensembl(ref_clean_sub_poolabcdf, lookup)
ref_clean_sub_poolabcd_ensembl = replace_w_ensembl(ref_clean_sub_poolabcd, lookup)
ref_clean_sub_poolf_ensembl = replace_w_ensembl(ref_clean_sub_poolf, lookup)

Number of rows with missing Ensembl mapping: 118
Gene symbols with no mapping:
['CTD2515O10' 'LOC541473' 'LOC284865' 'CTD2574D22' 'LOC100101148'
 'LOC388849' 'GREGOR' 'XXBACB562F10' 'LOC100133091' 'SEPT5GP1BB'
 'LOC101926943']
Number of rows with missing Ensembl mapping: 0
Gene symbols with no mapping:
[]
Number of rows with missing Ensembl mapping: 118
Gene symbols with no mapping:
['CTD2515O10' 'LOC541473' 'LOC284865' 'CTD2574D22' 'LOC100101148'
 'LOC388849' 'GREGOR' 'XXBACB562F10' 'LOC100133091' 'SEPT5GP1BB'
 'LOC101926943']


In [ ]:
# Write to file
ref_clean_sub_poolabcd.to_csv(local_path + "harmonized_guide_file_poolabcd_ensg.csv")
ref_clean_sub_poolabcd.to_csv(
    local_path + "harmonized_guide_file_poolabcd_ensg.tsv",
    sep='\t',
    index=False,
    lineterminator='\n'
)
ref_clean_sub_poolf.to_csv(local_path + "harmonized_guide_file_poolf_ensg.csv")
ref_clean_sub_poolf.to_csv(
    local_path + "harmonized_guide_file_poolf_ensg.tsv",
    sep='\t',
    index=False,
    lineterminator='\n'
)
ref_clean_sub_poolabcdf.to_csv(local_path + "harmonized_guide_file_poolabcdf_ensg.csv")

# Write to tsv file, including header
ref_clean_sub_poolabcdf.to_csv(
    local_path + "harmonized_guide_file_poolabcdf_ensg.tsv",
    sep='\t',
    index=False,
    lineterminator='\n'
)